In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from math import sqrt
import math
import os
import json
import time
import seaborn as sns
import datetime
from core.data_processor import DataLoader
from core.model import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:
'''
till now judgement
date_time: convert to suitable format, doesnt go for prediction 
dayofweek: add new column signifying day(Monday=0, Sunday=6)
peakhours: 
is_holiday : change all non null to 1
wind_speed : to be standardized variable
wind_direction: categorical variable, better to put in 60 deg buckets 
visibility_in_miles: categorical
dew_point: categorical
temperature: replace 0 with mean value and standardize it
air_pollution_index : No change, just standardize it
humidity: standardize it
rain_p_h: have to think , for now just use as it is
snow_ph: 0 and 1
weather_type: label encode
weather_description: for now label encode
clouds_all : 
traffic_volume: to predict
'''


'\ntill now judgement\ndate_time: convert to suitable format, doesnt go for prediction \ndayofweek: add new column signifying day(Monday=0, Sunday=6)\npeakhours: \nis_holiday : change all non null to 1\nwind_speed : to be standardized variable\nwind_direction: categorical variable, better to put in 60 deg buckets \nvisibility_in_miles: categorical\ndew_point: categorical\ntemperature: replace 0 with mean value and standardize it\nair_pollution_index : No change, just standardize it\nhumidity: standardize it\nrain_p_h: have to think , for now just use as it is\nsnow_ph: 0 and 1\nweather_type: label encode\nweather_description: for now label encode\nclouds_all : \ntraffic_volume: to predict\n'

In [3]:
rawdata = pd.read_csv("./DataSets/Train.csv")
df_test = pd.read_csv("./DataSets/Test.csv")

In [4]:
rawdata.head()

,date_time,is_holiday,air_pollution_index,humidity,wind_speed,wind_direction,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume
0,2012-10-02 09:00:00,None,121,89,2,329,1,1,288.28,0.0,0.0,40,Clouds,scattered clouds,5545
1,2012-10-02 10:00:00,None,178,67,3,330,1,1,289.36,0.0,0.0,75,Clouds,broken clouds,4516
2,2012-10-02 11:00:00,None,113,66,3,329,2,2,289.58,0.0,0.0,90,Clouds,overcast clouds,4767
3,2012-10-02 12:00:00,None,20,66,3,329,5,5,290.13,0.0,0.0,90,Clouds,overcast clouds,5026
4,2012-10-02 13:00:00,None,281,65,3,329,7,7,291.14,0.0,0.0,75,Clouds,broken clouds,4918


In [5]:
rawdata.date_time = pd.to_datetime(rawdata['date_time'])

In [24]:
def preprocessing(rawdata):
    df = rawdata.copy()
    #day of week, we have less traffic during saturdays or sundays
    df['dayofweek'] = rawdata.date_time.dt.dayofweek
    df['month'] = rawdata.date_time.dt.month
    #hours
    bins = [-0.1,6,12,18,24]
    labels = [0,1,2,3]
    df['hour']= pd.cut(rawdata['date_time'].dt.hour, bins=bins, labels=labels)
    #is_holiday
    df['is_holiday'] = (rawdata.is_holiday!='None')*1
    A = df.loc[df.is_holiday==1,'date_time']
    for items in A:
        df.loc[(df.date_time.dt.year==items.year) & (df.date_time.dt.month==items.month) & (df.date_time.dt.day==items.day),'is_holiday'] = 1
    #wind_direction
    bins = [-0.1, 60, 120, 180, 240, 300, 360]
    labels = [0,1,2,3,4,5]
    df['wind_direction']= pd.cut(rawdata['wind_direction'], bins=bins, labels=labels)
    #temperature
    df.loc[rawdata.temperature==0,'temperature'] = np.mean(rawdata.loc[rawdata.temperature!=0,'temperature'])
    #snow_p_h
    df['snow_p_h'] = (rawdata.snow_p_h!=0)*1
    #weather label encode
    #Encoding features
    lb_weather = LabelEncoder()
    df['weather_type'] = lb_weather.fit_transform(rawdata['weather_type'])
    lb_weather_des = LabelEncoder()
    df['weather_description'] = lb_weather_des.fit_transform(rawdata['weather_description'])
    #standardizing
    sc = MinMaxScaler()
    std_cols = ['temperature','air_pollution_index','humidity','wind_speed','clouds_all']
    sc_traffic = MinMaxScaler()
    df["traffic_volume"] = sc_traffic.fit_transform(np.array(rawdata["traffic_volume"]).reshape(-1,1))
#     mean = rawdata[std_cols].mean()
#     std = rawdata[std_cols].std()
    df[std_cols] = sc.fit_transform(rawdata[std_cols])
    return df,(lb_weather,lb_weather_des),(std_cols,sc,sc_traffic)

In [25]:
df,encoders,stats = preprocessing(rawdata)

In [26]:
df.describe()

,is_holiday,air_pollution_index,humidity,wind_speed,visibility_in_miles,dew_point,temperature,rain_p_h,snow_p_h,clouds_all,weather_type,weather_description,traffic_volume,dayofweek,month
count,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000,33750.000000
mean,0.028859,0.501181,0.669069,0.211137,4.989748,4.989748,0.908609,0.448739,0.001867,0.504588,2.520000,15.959170,0.445071,2.982667,6.407674
std,0.167413,0.289742,0.193704,0.128487,2.570021,2.570021,0.043522,53.526500,0.043165,0.388717,2.739005,9.081681,0.273556,2.005717,3.543241
min,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.252595,0.540230,0.125000,3.000000,3.000000,0.881521,0.000000,0.000000,0.010000,0.000000,7.000000,0.160062,1.000000,3.000000
50%,0.000000,0.501730,0.678161,0.187500,5.000000,5.000000,0.908870,0.000000,0.000000,0.640000,1.000000,17.000000,0.458104,3.000000,6.000000
75%,0.000000,0.754325,0.827586,0.312500,7.000000,7.000000,0.942837,0.000000,0.000000,0.900000,5.000000,24.000000,0.676648,5.000000,10.000000
max,1.000000,1.000000,1.000000,1.000000,9.000000,9.000000,1.000000,9831.300000,1.000000,1.000000,10.000000,37.000000,1.000000,6.000000,12.000000


In [27]:
df.columns

Index([u'date_time', u'is_holiday', u'air_pollution_index', u'humidity',
       u'wind_speed', u'wind_direction', u'visibility_in_miles', u'dew_point',
       u'temperature', u'rain_p_h', u'snow_p_h', u'clouds_all',
       u'weather_type', u'weather_description', u'traffic_volume',
       u'dayofweek', u'month', u'hour'],
      dtype='object')

In [28]:
#saving file for easy training purpose
print('Saving as CSV...')
df.to_csv('clean_train.csv',index=False)
df.info()

Saving as CSV...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33750 entries, 0 to 33749
Data columns (total 18 columns):
date_time              33750 non-null datetime64[ns]
is_holiday             33750 non-null int64
air_pollution_index    33750 non-null float64
humidity               33750 non-null float64
wind_speed             33750 non-null float64
wind_direction         33750 non-null category
visibility_in_miles    33750 non-null int64
dew_point              33750 non-null int64
temperature            33750 non-null float64
rain_p_h               33750 non-null float64
snow_p_h               33750 non-null int64
clouds_all             33750 non-null float64
weather_type           33750 non-null int64
weather_description    33750 non-null int64
traffic_volume         33750 non-null float64
dayofweek              33750 non-null int64
month                  33750 non-null int64
hour                   33750 non-null category
dtypes: category(2), datetime64[ns](1), float64(7), i

In [12]:
#running model
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

data = DataLoader(
    configs['data']['filename'],
    configs['data']['train_test_split'],
    configs['data']['columns']
)

model = Model()
model.build_model(configs)
model.load_model('./saved_models/28072019-004141-e50.h5')

# out-of memory generative training
steps_per_epoch = math.ceil((data.len_train - configs['data']['sequence_length']) / configs['training']['batch_size'])
model.train_generator(
    data_gen=data.generate_train_batch(
        seq_len=configs['data']['sequence_length'],
        batch_size=configs['training']['batch_size'],
        normalise=configs['data']['normalise']
    ),
    epochs=configs['training']['epochs'],
    batch_size=configs['training']['batch_size'],
    steps_per_epoch=steps_per_epoch,
    save_dir=configs['model']['save_dir']
)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 24, 30)            5760      
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 24, 40)            11360     
_________________________________________________________________
lstm_6 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_4 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 41        
Total params: 30,121
Trainable params: 30,121
Non-trainable params: 0
_________________________________________________________________
[Model

421/421 [==============================] - 83s 197ms/step - loss: 8107829.8680 - mean_absolute_error: 2268.0068
Epoch 54/150
421/421 [==============================] - 83s 197ms/step - loss: 8009978.4211 - mean_absolute_error: 2250.8579
Epoch 55/150
421/421 [==============================] - 84s 198ms/step - loss: 7950273.3306 - mean_absolute_error: 2239.7177
Epoch 56/150
421/421 [==============================] - 84s 199ms/step - loss: 7875406.7896 - mean_absolute_error: 2226.8105
Epoch 57/150
421/421 [==============================] - 85s 202ms/step - loss: 7821247.7623 - mean_absolute_error: 2222.6435
Epoch 58/150
421/421 [==============================] - 83s 197ms/step - loss: 7733566.1519 - mean_absolute_error: 2204.4345s - loss: 7764976.8849 - mean_absolut
Epoch 59/150
421/421 [==============================] - 83s 198ms/step - loss: 7684935.4193 - mean_absolute_error: 2198.7738
Epoch 60/150
421/421 [==============================] - 84s 199ms/step - loss: 7603082.6396 - mean_ab

StopIteration: 

In [29]:
#Loading model if you didnt train
#running model
configs = json.load(open('config.json', 'r'))
if not os.path.exists(configs['model']['save_dir']): os.makedirs(configs['model']['save_dir'])

data = DataLoader(
    configs['data']['filename'],
    configs['data']['train_test_split'],
    configs['data']['columns']
)

model = Model()
model.build_model(configs)
model.load_model('./saved_models/28072019-004141-e50.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 30)            5760      
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 24, 40)            11360     
_________________________________________________________________
lstm_3 (LSTM)                (None, 40)                12960     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 41        
Total params: 30,121
Trainable params: 30,121
Non-trainable params: 0
_________________________________________________________________
[Model

In [30]:
#Test data preperation
df_test.date_time = pd.to_datetime(df_test['date_time'])

In [37]:
def prepping_test(rawdata,stats,encoder):
    df = rawdata.copy()
    #day of week, we have less traffic during saturdays or sundays
    df['dayofweek'] = rawdata.date_time.dt.dayofweek
    df['month'] = rawdata.date_time.dt.month
    #hours
    bins = [-0.1,6,12,18,24]
    labels = [0,1,2,3]
    df['hour']= pd.cut(rawdata['date_time'].dt.hour, bins=bins, labels=labels)
    #is_holiday
    df['is_holiday'] = (rawdata.is_holiday!='None')*1
    A = df.loc[df.is_holiday==1,'date_time']
    for items in A:
        df.loc[(df.date_time.dt.year==items.year) & (df.date_time.dt.month==items.month) & (df.date_time.dt.day==items.day),'is_holiday'] = 1
    #wind_direction
    bins = [-0.1, 60, 120, 180, 240, 300, 360]
    labels = [0,1,2,3,4,5]
    df['wind_direction']= pd.cut(rawdata['wind_direction'], bins=bins, labels=labels)
    #temperature
    df.loc[rawdata.temperature==0,'temperature'] = np.mean(rawdata.loc[rawdata.temperature!=0,'temperature'])
    #snow_p_h
    df['snow_p_h'] = (rawdata.snow_p_h!=0)*1
    #weather label encode
    #Encoding features
    df['weather_type'] = encoder[0].transform(rawdata['weather_type'])
    df['weather_description'] = encoder[1].transform(rawdata['weather_description'])
    #normalizing, not including traffic volume
    std_cols = stats[0]
#     mean = stats[1][:-1]
#     std = stats[2][:-1]
    df[std_cols] = stats[1].transform(rawdata[std_cols])
    return df

In [38]:
df_test_prepped = prepping_test(df_test,stats,encoders)
df_test_prepped["traffic_volume"] = np.NaN
data_to_attach=configs["data"]["sequence_length"]-1
df_test_prepped=pd.concat([df.iloc[-data_to_attach:],df_test_prepped],axis=0)
df_test_prepped.reset_index(drop=True)
df_test_prepped.to_csv("./clean_test.csv",index=False)

In [ ]:
def next_window(data_train,i, seq_len):
    '''Generates the next data window from the given index location i'''
    window = data_train[i:i+seq_len]
    x = window[:-1]
    y = window[-1, [0]]
    return x, y

In [ ]:
#I generate data window, preditict and update and repeat
for i in range(len(df_test_prepped)-configs['data']['sequence_length'] + 1):
    print("Done {}/{}".format(i,len(df_test_prepped)-configs['data']['sequence_length'])+1)
    x_test,y = next_window(df_test_prepped.get(configs["data"]["columns"]).values,i,configs['data']['sequence_length'])    
    pred = model.model.predict(x_test[np.newaxis])[0][0]
    df_test_prepped.traffic_volume[24+i] = pred

In [ ]:
#de normalizing 
df_test_prepped.traffic_volume = stats[2].inverse_transform(np.array(df_test_prepped.traffic_volume).reshape(-1,1)